#  High-thoughput ab initio calculation with Python

Tutorial para o evento **Machine Learning School for Materials**, Ilum, CNPEM, 2022.

Henrique Ferreira dos Santos (hfsantos@ufabc.edu.br)

---------------------------------

# Parte 1 - Screening 

Antes de iniciar as contas **High Throughput** vamos obter informações de bases de dados de contas DFT.

O **screening** é o procedimento de busca e seleção de materiais nessas bases de dados, de maneira automatizada e baseado em critérios estebelecidos pelo pesquisador.

Como exemplo, vamos realizar um estudo de semicontudores de **Ultrawide Bandgap** (UWBG semiconductors). Inicialmente vamos buscar nas bases de dados compostos químicos que já foram simulados e apresentam gap ultra largo. Em seguida, vamos filtrá-los com uma série de critérios que iremos estabelecer. Por fim, selecionaremos um composto da lista final para realizar um estudo mais detalhado que será feito na Parte 2 desse tutorial.


### Bibliotecas utilizadas

In [ ]:
#!pip install pymatgen  # Caso não tenha a biblioteca pymatgen, você pode instalá-la usando este comando

In [ ]:
# Pymatgen is a open-source librarie for materials analysis
from pymatgen.ext.matproj  import MPRester                         # API requester for Materials project
from pymatgen.core.periodic_table import Element                   # Class to represent Element in pymatgen
from pymatgen.io import vasp                                       # Interface with VASP

import pandas as pd   # Working with tables

import os
import time

## Critérios de busca

- [ ] Materiais ternários
- [ ] Elementos não radioativos
- [ ] Bandgap entre 4 e 12 eV
- [ ] Têm gap direto
- [ ] Estrutura de bandas foi reportada
- [ ] Tem entrada no ICSD
- [ ] Materiais termodinamicamente estáveis (estão no convex hull)
- [ ] Tem pelo menos uma rota de síntese conhecida

### Elementos não radioativos

In [ ]:
# Gerando a lista de elementos não radioativos

def desired_element(elem):
    omit = ['Po', 'At', 'Rn', 'Fr', 'Ra']
    return not elem.is_noble_gas and not elem.is_actinoid and not elem.symbol in omit

element_universe = [e for e in Element if desired_element(e)]
omitted_elements = [e for e in Element if e not in element_universe]
elements = [e.symbol for e in element_universe] 

print("Número de elementos incluídos =", len(element_universe))
print("Elementos excluídos: ", " ".join(sorted([e.symbol for e in omitted_elements])))

### Base de busca - Materials Project
O Materials Project é um base de dados pública. 

Os critérios de busca do Materials Project são limitados a uma lista de propriedades que pode ser consultada aqui https://workshop.materialsproject.org/lessons/04_materials_api/MAPI%20Lesson%20%28filled%29/.

O Materials Project é feito usando tecnologia MongoDB para o banco de dados. Dessa forma, a query pode ser aperfeiçoada usando a sintaxe desse banco de dados. A sintaxe pode ser consultada em https://www.mongodb.com/docs/manual/reference/operator/query/

As propriedades que são possiveis de serem capturadas estão em https://github.com/materialsproject/mapidoc/materials. A princípio, todas as pastas são informações que podem ser acessadas. Entretanto, nem todas as informações podem ser acessadas em uma query geral (precisam ser acessadas via query específica para um material) e algumas delas podem não estar presentes (faltam para aquele material).

In [ ]:
# Critérios de busca na base de dados
criteria = {'nelements':{'$in': [3]},            # Somente materiais ternários
            'band_gap':{'$gte': 4, '$lte': 12},  # Bandgap está entre 4 e 12 (gte >= and lte <=)
            'elements':{'$in':elements},         # Lista de elementos permitidos
            '$where':'this.icsd_ids.length>0',   # Tem entrada no ICSD
            'band_gap.search_gap.is_direct': {'$eq': True}, # Bandgap direto
            'has_bandstructure':{'$eq': True}    # Estrutura de bandas foi calculada
           } 

# Propriedades buscadas
properties =['material_id', 'icsd_ids', 'pretty_formula','elements', 'band_gap','formation_energy_per_atom',
             'e_above_hull', 'spacegroup']

# Chave de acesso
apikey = ''

In [ ]:
# Chamada API rest para requisitar os dados ao servidor do MP
with MPRester(apikey) as mpr:
    results = mpr.query(criteria, properties)

Armazenamos os resultados da requisição (488 materiais) na variável <code>result</code>.

Para facilitar o trabalho com essa variável, vamos transformá-la em uma tabela via pandas do tipo <code>DataFrame</code>:

In [ ]:
mat_list0 = pd.DataFrame(data = results)

In [ ]:
type(mat_list0)

In [ ]:
len(mat_list0) # Quantidade de entradas retornadas

In [ ]:
mat_list0.head(10) # Olhando as 10 primeiras linhas da tabela de materiais

A coluna <code>spacegroup</code> é do tipo dicionário (dict). Vamos transformar cada chave do dicionário em uma nova coluna:

In [ ]:
# Rearranja as informações dos grupos espaciais
mat_list0[['symprec',
          'source',
          'symbol',
          'number',
          'point_group',
          'crystal_system',
          'hall']] = mat_list0.spacegroup.apply(pd.Series)
mat_list0 = mat_list0.drop('spacegroup', axis=1)

In [ ]:
mat_list0.head()

### Estatísticas Básicas

Pelos nossos filtros de busca, a solicitação para o banco de dados retornou apenas materiais que respeitavam os seguintes critérios:

- [x] Materiais ternários
- [x] Elementos não radioativos
- [x] Bandgap entre 4 e 12 eV
- [x] Têm gap direto
- [x] Estrutura de bandas foi reportada
- [x] Tem entrada no ICSD
- [ ] Materiais termodinamicamente estáveis (estão no convex hull)
- [ ] Tem pelo menos uma rota de síntese conhecida
Vamos visualizar algumas informações estatísticas básicas dos compostos que temos até o momento:

In [ ]:
mat_list0.describe() # Gera informações estatísticas das colunas numéricas

In [ ]:
mat_list0.hist('band_gap')  # Plota o histograma do bandgap

In [ ]:
mat_list0.crystal_system.value_counts().plot(kind='bar')  # Distribuição dos sistemas cristalinos

### Materiais Estáveis
Agora vamos filtrar apenas os estáveis:

In [ ]:
mat_list1 = mat_list0[mat_list0['e_above_hull']==0]

In [ ]:
mat_list1.describe()

### Rotas de Síntese

Vamos verificar se existem rotas de sintese conhecidas para esses materiais. Para fazer isso precisaremos de dados adicionais que não existem no Materials Project. Aqui vamos utilizar a base de dados disponibilizada em https://github.com/CederGroupHub/text-mined-synthesis_public. Essa base de dados foi levantada usando-se ferramentas de Processamento de Linguagem Natural em cima de diversos artigos científicos.

No link, existem três arquivos <code>.json</code> comprimidos no formato <code>.xz</code>. Vamos escolher o  <code>solid-state_dataset_2019-12-03.json.xz</code>. Devemos baixá-lo e descompactá-lo. Em seguida, caso o ambiente usado seja o Google Colab, você deverá subir o arquivo na pasta lateral esquerda, ou usar o seguinte comando para baixar o arquivo e descompactá-lo automaticamente (atenção, este comando funciona melhor em ambientes Linux, como o Google Colab - se você estiver rodando na sua máquina local com Windows, considere abrir o arquivo diretamente pulado a próxima célula):

In [ ]:
!wget https://github.com/CederGroupHub/text-mined-synthesis_public/raw/master/solid-state_dataset_2019-12-03.json.xz && xz -d solid-state_dataset_2019-12-03.json.xz

In [ ]:
synth_data = pd.read_json('solid-state_dataset_2019-12-03.json')

In [ ]:
synth_data.head()

In [ ]:
len(synth_data) # Quantidade de entradas

Como as informações que vieram estão em duas colunas, vamos olhar quais são os atributos do objeto <code>dict</code> que estão na coluna <code>reactions</code>.

In [ ]:
synth_data['reactions'][0].keys()

Da mesma forma que fizemos para o grupo espacial, podemos fazer para este caso, criando uma nova coluna para cada atributo (chave) do dicionário:

In [ ]:
# Rearranja as informações 
synth_data[list(synth_data['reactions'][0].keys())] = synth_data.reactions.apply(pd.Series)
synth_data = synth_data.drop('reactions', axis=1) # Deletando a coluna reactions original

In [ ]:
synth_data.head()

Como queremos saber se o composto alvo da síntese está na nossa lista de compostos UWBG iremos arrumar a coluna <code>target</code>: 

In [ ]:
synth_data['target'][0].keys()

In [ ]:
# Rearranja as informações 
synth_data[list(synth_data['target'][0].keys())] = synth_data.target.apply(pd.Series)
synth_data = synth_data.drop('target', axis=1)

In [ ]:
synth_data.head()

Agora vamos usar a coluna <code>mp_id</code> que contém um identificador de entrada no Materials Project para comparar os dois:

In [ ]:
mat_list2 = mat_list1.merge(synth_data, left_on='material_id', right_on='mp_id')

In [ ]:
mat_list2.head()

Neste ponto podemos ter mais de uma linha para o mesmo composto caso haja mais de um rota de síntese na base de dados usada. Observe que na tabela dos compostos do MP haviamos encotnrado apenas 249, e que acgoram temos 340 entradas (mais de uma por material!). Podemos querer uma versão reduzida somente com os compostos únicos:

In [ ]:
len(mat_list2) # Total de linhas (podendo conter duplicidades devido a mais de uma rota de síntese)

In [ ]:
mat_list3 = mat_list2.drop_duplicates(subset=['material_id'])

In [ ]:
len(mat_list3) # Lista reduzida com compostos únicos

Dessa forma, concluímos que apenas 38 materiais atendem os critérios estabelecidos, levando-se em consideração as duas bases de dados utilizadas:

- [x] Materiais ternários
- [x] Elementos não radioativos
- [x] Bandgap entre 4 e 12 eV
- [x] Têm gap direto
- [x] Estrutura de bandas foi reportada
- [x] Tem entrada no ICSD
- [x] Materiais termodinamicamente estáveis (estão no convex hull)
- [x] Tem pelo menos uma rota de síntese conhecida

In [ ]:
mat_list3['pretty_formula'].values

Vamos escolher um desses materiais para estudar em detalhe: **CaAlF5**

In [ ]:
selected_material = mat_list2[mat_list2['pretty_formula']=='CaAlF5'] # Estamos pegando na lista 2 com todas as rotas

In [ ]:
selected_material

In [ ]:
selected = 'mp-8836' # ID do Materials Project do material escolhido

In [ ]:
selected_material.keys() # Propriedades nas tabelas manipuladas

### Rota de Síntese

In [ ]:
selected_material['reaction_string'].values

In [ ]:
selected_material['operations'].values

### Estrutura Eletrônica

In [ ]:
with MPRester(apikey) as mpr:
    bs = mpr.get_bandstructure_by_material_id(selected)

In [ ]:
type(bs)

In [ ]:
bs.get_band_gap()

In [ ]:
from pymatgen.electronic_structure.plotter import BSPlotter

In [ ]:
efermi=bs.efermi
print(efermi)  # Energia de Fermi eV

In [ ]:
bsp = BSPlotter(bs)
bsp.get_plot(zero_to_efermi=True).show() # Plote automatico do Pymatgen
#bsp.bs_plot_data(zero_to_efermi=True)   # Pega os dados em forma de dicionário para fazer o plot manual

### Estrutura Cristalina

In [ ]:
with MPRester(apikey) as mpr:
    structure = mpr.get_structure_by_material_id(selected)

In [ ]:
structure

In [ ]:
# OBS: A variável bs que armazena a estrutura eletrônica, também armazena a estrutura cristalina
#bs.structure

Vamos salvar essa estrutura em um arquivo <code>POSCAR</code> dentro de uma pasta com o nome do composto <code>proto_CaAlF5</code>, que ficará dentro de uma pasta geral <code>ht</code>. 

In [ ]:
poscar = vasp.inputs.Poscar(structure)

In [ ]:
master_folder = 'ht'
material_folder = 'proto_CaAlF5'
filename = 'POSCAR'

In [ ]:
os.mkdir(master_folder)
os.mkdir(master_folder+'/'+material_folder)

In [ ]:
poscar.write_file(master_folder+'/'+material_folder+'/'+filename)

Uma pergunta interessante é: <font color='red'>se trocarmos cada elemento desse composto por outro elemento com alta semelhança química, ainda obtemos um UWBG estável e sintetizável?</font> 

Visando responder isso, vamos realizar a Parte 2 desse tutorial, onde definiremos simulações DFT de novos materiais.

-------------------------------------------------

# Parte 2 - Configurando as entradas para contas HT 

Nesta etapa, já temos a pergunta e o objeto de estudo oriundas da Parte 1: CaAlF5 (mp-8836)

Vamos estabelecer que a similaridade química será definida por elementos do mesmo grupo químico. Vamos desconsiderar o Rádio (Ra) e o Ástato (At).

Como *solver* de Mecânica Quântica, vamos utilizar o software VASP.

Estamos assumindo que as contas serão realizadas em um cluster com gerenciador de fila (como o SLURM) e que não iremos utilizar nenhuma biblioteca adicional para auxiliar no gerenciamento das contas, exceto aquelas de pré-processamento (Parte 2) e pós-processamento (Parte 3).

Em caso de uso de clusters sem fila, sugere-se uso da biblioteca ASE (https://wiki.fysik.dtu.dk/ase/) para submissão de cálculos direto pelo código Python. Para usuários mais avançados que requeiram altas vazões de contas, inclusive em clusters com gereciamento de fila, sugere-se o uso do AiiDA (https://www.aiida.net/).

In [ ]:
from pymatgen.io import vasp                                       # Interface with VASP
from pymatgen.core.structure import Structure                      # Class to represent Structures in pymatgen
from pymatgen.core.composition import Composition                  # Class to represent Composition in pymatgen
from pymatgen.core.periodic_table import Element                   # Class to represent Element in pymatgen
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer          # Methods to analyze space groups of materials

import shutil
import os

### Criando novos materiais e configurando arquivos VASP

Nessa primeira etapa estamos interessados em fazer a atualização das posições ionicas para encontrar as estruturas com menor enegia (convergidas).

Vamos utilizar a estrutura original do CaAlF5 como protótipo e manter a simetria, trocando apenas os átomos dos elementos da mesma coluna da tabela periódica.

Primeiro vamos carregar nosso protótipo:

In [ ]:
input_folder = 'ht/proto_CaAlF5'  # Pasta onde está a estrutura inicial (POSCAR) usada como protótipo
proto_poscar = Structure.from_file(input_folder+"/POSCAR")  # Estrutura protótipo

In [ ]:
proto_space_group = SpacegroupAnalyzer(proto_poscar, symprec=0.1, angle_tolerance=1.0)
proto_space_group.get_space_group_number()

Vamos usar as seguintes configurações de arquivos VASP:

In [ ]:
incar_rx = {
"ALGO": "Normal",
"EDIFF": "0.000001",
"ENCUT": 520,
"IBRION": 2,
"ISIF": 3,
"ISMEAR": 0,
"ISPIN": 1,
"LASPH": True,
"LORBIT": 11,
"LREAL": "Auto",
"LWAVE": False,
"LCHARG":False,
"NELM": 100,
"NSW": 900,
"SIGMA": 0.05,
"NPAR": 6
}

auto_kpoints = 25

Agora iremos criar a estrutura de pasta com todos os arquivos necessários para submeter as contas e todos os novos compostos:

In [ ]:
# Lista de elementos que serão usados para fazer as substituições químicas
earthalk_list = ['Be','Mg','Ca','Sr','Ba']
boron_list = ['B','Al','Ga','In','Tl']
halogen_list = ['F','Cl','Br','I']

In [ ]:
calculations_folder = 'ht/rx'
os.mkdir(calculations_folder)

for e1 in earthalk_list:
    for e2 in boron_list:
        for e3 in halogen_list:
            
            # Nova pasta para o novo material em disco
            output_folder=calculations_folder+'/'+e1+e2+e3+'5'
            os.mkdir(output_folder)
            
            temp_structure = proto_poscar.copy()          # Cópia da estrutura original em memória

            if not(e1=='Ca' and e2=='Al' and e3=='F'):
                for i, element in enumerate(proto_poscar):    # Para dada atomo na estrutura
                    element_str = str(element.species)        # Pega o atomo no sítio
                    element_str = element_str.replace('1','') # Remove o número de estequiometria (no caso o número é 1)

                    # Troca apenas elementos da mesma familia
                    if element_str in earthalk_list:
                        temp_structure.replace(i,e1)
                    elif element_str in boron_list:
                        temp_structure.replace(i,e2)
                    elif element_str in halogen_list:
                        temp_structure.replace(i,e3)

            # Aviso de que a eventualmente uma estrutura não manteve a simetria pretendida
            space_group = SpacegroupAnalyzer(temp_structure, symprec=0.1, angle_tolerance=1.0)
            if not(space_group.get_space_group_number()==proto_space_group.get_space_group_number()):
                print('Warning: Estrutura '+e1+e2+e3+'5 com grupo espacial diferente do protótipo!') 

            # Criar o novo POSCAR na pasta de destino
            new_poscar = vasp.inputs.Poscar(temp_structure)
            new_poscar.write_file(output_folder+'/POSCAR')
            
            # Criar o arquivo KPOINTS na pasta de destino
            kpoints = vasp.Kpoints().automatic(auto_kpoints)  # Método automatico com auto_kpoints (25) pontos
            kpoints.write_file(output_folder+'/KPOINTS')
            
            # Criar arquivo INCAR na pasta de destino
            incar = vasp.Incar(incar_rx)
            incar.write_file(output_folder+'/INCAR')
            
            # Criar arquivo POTCAR na pasta de destino
            
            
            # Criar arquivo JOB na pasta de destino
            

# Parte 3 - Submentendo as contas para o solver de MQ

# Parte 4 - Analisando resultados